In [10]:
pip install requests

In [ ]:
    "꽃게탕", "홍합탕", "해물누룽지탕", "된장찌개", "차돌된장찌개", "꽃게된장찌개", "김치찌개", "순두부찌개",
    "부대찌개", "청국장", "동태찌개", "비지찌개", "고추장찌개", "짜글이찌개", "버섯찌개", "소고기찌개",
    "새우찌개", "밀폐유나베", "소고기버섯전골", "불고기전골", "어묵전골", "만두전골", "두부전골", "버섯전골",
    "곱창전골", "대창전골", "아롱사태전골", "불낙전골", "삼겹살", "수육", "스테이크구이", "찹스테이크",
    "갈비찜", "돼지갈비", "LA갈비", "바베큐", "김치등갈비찜", "묵은지돼지갈비찜", "매운쪽갈비찜", "폭립",
    "제육볶음","소불고기", "돼지불고기", "닭볶음탕", "닭갈비", "훈제오리구이", "단호박훈제오리찜", "삼겹살숙주볶음",
    "차돌숙주볶음", "찜닭", "소세지야채볶음", "돈까스", "떡갈비", "함박스테이크", "동그랑땡", "편백나무찜",
    "곱창구이", "막창구이", "족발", "치킨", "닭강정", "생선까스", "연어스테이크", "오징어볶음",
    "미나리오징어초무침", "쭈꾸미볶음", "아귀찜", "해물찜", "고등어구이", "고등어조림", "코다리조림",
    "갈치구이", "갈치조림", "장어구이", "조기구이", "가자미구이", "꽁치조림", "낙곱새", "바지락술찜",
    "바지락칼국수", "꽃게찜", "대게찜", "랍스터찜", "간장게장", "양념게장", "새우장", "회", "생굴",
    "잡채", "골뱅이무침","마늘빵", "토스트", "냉면", "잔치국수", "비빔국수", "열무국수", "콩국수", "수제비", "비빔만두", "쫄면",
    "칼국수", "떡국", "만둣국", "우동", "핫도그", "찐만두", "튀김만두", "비빔만두", "김말이튀김", "야채튀김",
    "오징어튀김", "가지튀김", "팝콘치킨", "순대", "짜장면", "짬뽕", "마라탕", "탕수육", "꿔바로우", "마파두부",
    "계란토마토볶음밥", "양장피", "깐풍기", "깐풍새우", "크림새우", "유린기", "팔보채", "고추잡채", "춘권",
    "딤섬", "토마토스파게티", "크림파스타", "명란파스타", "봉골레파스타", "감바스", "스테이크", "또띠아피자",
    "고구마그라탕", "감자그라탕", "피자", "함박스테이크", "리조또", "스테이크", "샐러드", "햄버거", "부리또",
    "해쉬브라운", "감자튀김", "맥앤치즈", "콘샐러드", "초밥", "라멘", "나가사키짬뽕", "오니기리", "연어덮밥",
    "새우장덮밥", "메밀소바", "돈카츠", "야키니쿠", "낫또", "볶음우동", "카레우동", "가츠동", "규동", "쌀국수", "팟타이", "타코"

In [5]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time  # time 모듈 추가

# www.10000recipe.com에서 음식 정보를 가져오는 함수
def food_info(name, max_retries=3):
    url = f"https://www.10000recipe.com/recipe/list.html?q={name}"

    for attempt in range(max_retries):
        try:
            response = requests.get(url)
            response.raise_for_status()  # 오류가 발생하면 예외를 일으킴
            break  # 성공했으므로 반복문 종료
        except requests.HTTPError as e:
            print(f"HTTP 응답 오류 ({response.status_code}): {e}")
            if attempt < max_retries - 1:
                print(f"재시도 {attempt + 1}/{max_retries}...")
                time.sleep(2)  # 일정 시간 대기 후 재시도
            else:
                print(f"{max_retries}번의 재시도 후에도 실패했습니다. 프로그램 종료.")
                return

    soup = BeautifulSoup(response.text, 'html.parser')
    food_list = soup.find_all(attrs={'class': 'common_sp_link'})

    if not food_list:
        print(f"'{name}'에 대한 레시피를 찾을 수 없습니다.")
        return

    food_id = food_list[0]['href'].split('/')[-1]
    new_url = f'https://www.10000recipe.com/recipe/{food_id}'

    for attempt in range(max_retries):
        try:
            new_response = requests.get(new_url)
            new_response.raise_for_status()
            break
        except requests.HTTPError as e:
            print(f"HTTP 응답 오류 ({new_response.status_code}): {e}")
            if attempt < max_retries - 1:
                print(f"재시도 {attempt + 1}/{max_retries}...")
                time.sleep(2)
            else:
                print(f"{max_retries}번의 재시도 후에도 실패했습니다. 프로그램 종료.")
                return

    soup = BeautifulSoup(new_response.text, 'html.parser')
    food_info = soup.find(attrs={'type': 'application/ld+json'})

    if not food_info:
        print(f"'{name}'에 대한 레시피 정보를 찾을 수 없습니다.")
        return

    result = json.loads(food_info.text)

    recipe = [step["text"] for step in result.get('recipeInstructions', [])]
    joined_recipe = "\n".join(recipe)

    return {
        'name': name,
        'recipe': joined_recipe
    }

# Food list
food_list = [
    "잡곡밥", "옥수수밥", "감자밥", "무밥", "명란버터밥", "비빔밥", "가지밥", "전복밥", "콩나물밥", "곤드레비빔밥",
    "표고버섯영양밥", "쌈밥", "야채죽", "전복죽", "새우죽", "삼계죽", "미역죽", "참치죽", "소고기버섯죽", "팥죽",
    "단호박죽", "베이컨볶음밥", "김치볶음밥", "간장계란밥", "소고기볶음밥", "스팸볶음밥", "해물볶음밥", "새우볶음밥",
    "카레덮밥", "짜장밥", "오징어덮밥", "오므라이스", "육회비빔밥", "김치알밥", "미역국", "무국", "콩나물국",
    "김치콩나물국", "사골곰탕", "북엇국", "우거지국", "시래기국", "배추된장국", "매생이국", "올갱이국", "뼈해장국",
    "된장국", "계란감자국", "계란국", "감자국", "오징어무국", "어묵탕", "육개장", "갈비탕", "삼계탕", "추어탕",
    "꽃게탕", "홍합탕", "해물누룽지탕", "된장찌개", "차돌된장찌개", "꽃게된장찌개", "김치찌개", "순두부찌개",
    "부대찌개", "청국장", "동태찌개", "비지찌개", "고추장찌개", "짜글이찌개", "버섯찌개", "소고기찌개",
    "새우찌개", "밀폐유나베", "소고기버섯전골", "불고기전골", "어묵전골", "만두전골", "두부전골", "버섯전골",
    "곱창전골", "대창전골", "아롱사태전골", "불낙전골", "삼겹살", "수육", "스테이크구이", "찹스테이크",
    "갈비찜", "돼지갈비", "LA갈비", "바베큐", "김치등갈비찜", "묵은지돼지갈비찜", "매운쪽갈비찜", "폭립",
    "제육볶음","소불고기", "돼지불고기", "닭볶음탕", "닭갈비", "훈제오리구이", "단호박훈제오리찜", "삼겹살숙주볶음",
    "차돌숙주볶음", "찜닭", "소세지야채볶음", "돈까스", "떡갈비", "함박스테이크", "동그랑땡", "편백나무찜",
    "곱창구이", "막창구이", "족발", "치킨", "닭강정", "생선까스", "연어스테이크", "오징어볶음",
    "미나리오징어초무침", "쭈꾸미볶음", "아귀찜", "해물찜", "고등어구이", "고등어조림", "코다리조림",
    "갈치구이", "갈치조림", "장어구이", "조기구이", "가자미구이", "꽁치조림", "낙곱새", "바지락술찜",
    "바지락칼국수", "꽃게찜", "대게찜", "랍스터찜", "간장게장", "양념게장", "새우장", "회", "생굴",
    "잡채", "골뱅이무침","마늘빵", "토스트", "냉면", "잔치국수", "비빔국수", "열무국수", "콩국수", "수제비", "비빔만두", "쫄면",
    "칼국수", "떡국", "만둣국", "우동", "핫도그", "찐만두", "튀김만두", "비빔만두", "김말이튀김", "야채튀김",
    "오징어튀김", "가지튀김", "팝콘치킨", "순대", "짜장면", "짬뽕", "마라탕", "탕수육", "꿔바로우", "마파두부",
    "계란토마토볶음밥", "양장피", "깐풍기", "깐풍새우", "크림새우", "유린기", "팔보채", "고추잡채", "춘권",
    "딤섬", "토마토스파게티", "크림파스타", "명란파스타", "봉골레파스타", "감바스", "스테이크", "또띠아피자",
    "고구마그라탕", "감자그라탕", "피자", "함박스테이크", "리조또", "스테이크", "샐러드", "햄버거", "부리또",
    "해쉬브라운", "감자튀김", "맥앤치즈", "콘샐러드", "초밥", "라멘", "나가사키짬뽕", "오니기리", "연어덮밥",
    "새우장덮밥", "메밀소바", "돈카츠", "야키니쿠", "낫또", "볶음우동", "카레우동", "가츠동", "규동", "쌀국수", "팟타이", "타코"

]

# Data storage
data_food_name, data_food_recipe = [], []

for food_name in food_list:
    food_data = food_info(food_name)
    if food_data:
        data_food_name.append(food_data['name'])
        data_food_recipe.append(food_data['recipe'])

# DataFrame 생성 및 CSV로 저장
food_dataframe = {
    'food_name': data_food_name,
    'food_recipe': data_food_recipe,
}

df = pd.DataFrame(food_dataframe)
df.to_csv('ONLY_food_recipe_dataframe.csv', index=False)
print("ONLY_food_recipe_dataframe.csv가 만들어졌습니다.")


ONLY_food_recipe_dataframe.csv가 만들어졌습니다.


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# www.10000recipe.com에서 음식 정보를 가져오는 함수
def food_info(name, max_retries=3, retry_interval=5):
    for attempt in range(max_retries):
        url = f"https://www.10000recipe.com/recipe/list.html?q={name}"
        response = requests.get(url)

        if response.status_code == 200:
            break
        else:
            print(f"HTTP 응답 오류 ({response.status_code}). 재시도 중...")
            time.sleep(retry_interval)

    if response.status_code != 200:
        print("HTTP 응답 오류:", response.status_code)
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    food_list = soup.find_all(attrs={'class': 'common_sp_link'})

    if not food_list:
        print(f"'{name}'에 대한 레시피를 찾을 수 없습니다.")
        return None

    food_id = food_list[0]['href'].split('/')[-1]
    new_url = f'https://www.10000recipe.com/recipe/{food_id}'

    for attempt in range(max_retries):
        new_response = requests.get(new_url)

        if new_response.status_code == 200:
            break
        else:
            print(f"HTTP 응답 오류 ({new_response.status_code}). 재시도 중...")
            time.sleep(retry_interval)

    if new_response.status_code != 200:
        print("HTTP 응답 오류:", new_response.status_code)
        return None

    soup = BeautifulSoup(new_response.text, 'html.parser')
    food_info = soup.find(attrs={'type': 'application/ld+json'})

    if not food_info:
        print(f"'{name}'에 대한 레시피 정보를 찾을 수 없습니다.")
        return None

    result = json.loads(food_info.text)

    # 난이도 크롤링 후 숫자로 변환
    menu_info_3_tag = soup.find('span', 'view2_summary_info3')  # nando 인덱스 추가 => 난이도
    nando = menu_info_3_tag.get_text() if menu_info_3_tag else "None"
    if nando == "아무나" :
        nando = 1
    elif nando in ["중급", "고급", "신의경지"]:
        nando = 3
    elif nando == "초급"
        nando = 2  # 기본값

    return {
        'name': name,
        'nando': nando
    }

# Food list
food_list = [
    "잡곡밥", "옥수수밥", "감자밥", "무밥"
]

# Data storage
data_food_name, data_nando = [], []

for food_name in food_list:
    food_data = food_info(food_name)
    if food_data:
        data_food_name.append(food_name)
        data_nando.append(food_data['nando'])

# DataFrame 생성 및 CSV로 저장
food_dataframe = {
    'food_name': data_food_name,
    'nando': data_nando
}

# df = pd.DataFrame(food_dataframe)
# df.to_csv('food_dataframe.csv', index=False)
# print("food_dataframe.csv가 만들어졌습니다.")


In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import json

# 양과 재료를 분리하는 함수
def separate_amount_ingredient(input_string):
    parts = input_string.split(',')
    ingredient_name_list, amount_list = [], []

    for part in parts:
        ingredient_amount = part.split(' ', 1)
        ingredient_name = ingredient_amount[0].strip()
        amount = ingredient_amount[1].strip() if len(ingredient_amount) > 1 else "None"
        ingredient_name_list.append(ingredient_name)
        amount_list.append(amount)

    return ingredient_name_list, amount_list

# www.10000recipe.com에서 음식 정보를 가져오는 함수
def food_info(name, max_retries=3, retry_interval=5):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    for attempt in range(max_retries):
        try:
            url = f"https://www.10000recipe.com/recipe/list.html?q={name}"
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            food_list = soup.find_all(attrs={'class': 'common_sp_link'})

            if not food_list:
                print(f"'{name}'에 대한 레시피를 찾을 수 없습니다.")
                return None

            food_id = food_list[0]['href'].split('/')[-1]
            new_url = f'https://www.10000recipe.com/recipe/{food_id}'

            new_response = requests.get(new_url, headers=headers, timeout=10)
            new_response.raise_for_status()

            soup = BeautifulSoup(new_response.text, 'html.parser')
            food_info = soup.find(attrs={'type': 'application/ld+json'})

            if not food_info:
                print(f"'{name}'에 대한 레시피 정보를 찾을 수 없습니다.")
                return None

            result = json.loads(food_info.text)

            ingredient = ','.join(result.get('recipeIngredient', []))
            ingredient_name_list, amount_list = separate_amount_ingredient(ingredient)

            cooking_time_tag = soup.find('span', {'class': 'view2_summary_info2'})
            cooking_time = cooking_time_tag.get_text(strip=True) if cooking_time_tag else "None"

            # 난이도 크롤링 후 숫자로 변환
            menu_info_3_tag = soup.find('span', 'view2_summary_info3')  # nando 인덱스 추가 => 난이도
            nando = menu_info_3_tag.get_text() if menu_info_3_tag else "None"
            if nando == "아무나":
                nando = 1
            elif nando in ["중급", "고급", "신의경지"]:
                nando = 3
            elif nando == "초급":
                nando = 2  # 기본값

            return {
                'name': name,
                'ingredient_name_list': ingredient_name_list,
                'amount_list': amount_list,
                'cooking_time': cooking_time,
                'nando': nando
            }

        except requests.exceptions.RequestException as e:
            print(f"에러 발생: {e}. 재시도 중...")
            time.sleep(retry_interval)

    print(f"다수의 시도 끝에 '{name}' 정보를 가져오는 데 실패했습니다.")
    return None

# Food list
food_list = [
    "잡곡밥", "옥수수밥", "감자밥", "무밥", "명란버터밥", "비빔밥", "가지밥", "전복밥", "콩나물밥", "곤드레비빔밥",
    "표고버섯영양밥", "쌈밥", "야채죽", "전복죽", "새우죽", "삼계죽", "미역죽", "참치죽", "소고기버섯죽", "팥죽",
    "단호박죽", "베이컨볶음밥", "김치볶음밥", "간장계란밥", "소고기볶음밥", "스팸볶음밥", "해물볶음밥", "새우볶음밥",
    "카레덮밥", "짜장밥", "오징어덮밥", "오므라이스", "육회비빔밥", "김치알밥", "미역국", "무국", "콩나물국",
    "김치콩나물국", "사골곰탕", "북엇국", "우거지국", "시래기국", "배추된장국", "매생이국", "올갱이국", "뼈해장국",
    "된장국", "계란감자국", "계란국", "감자국", "오징어무국", "어묵탕", "육개장", "갈비탕", "삼계탕", "추어탕",
    "꽃게탕", "홍합탕", "해물누룽지탕", "된장찌개", "차돌된장찌개", "꽃게된장찌개", "김치찌개", "순두부찌개",
    "부대찌개", "청국장", "동태찌개", "비지찌개", "고추장찌개", "짜글이찌개", "버섯찌개", "소고기찌개",
    "새우찌개", "밀폐유나베", "소고기버섯전골", "불고기전골", "어묵전골", "만두전골", "두부전골", "버섯전골",
    "곱창전골", "대창전골", "아롱사태전골", "불낙전골", "삼겹살", "수육", "스테이크구이", "찹스테이크",
    "갈비찜", "돼지갈비", "LA갈비", "바베큐", "김치등갈비찜", "묵은지돼지갈비찜", "매운쪽갈비찜", "폭립",
    "제육볶음","소불고기", "돼지불고기", "닭볶음탕", "닭갈비", "훈제오리구이", "단호박훈제오리찜", "삼겹살숙주볶음",
    "차돌숙주볶음", "찜닭", "소세지야채볶음", "돈까스", "떡갈비", "함박스테이크", "동그랑땡", "편백나무찜",
    "곱창구이", "막창구이", "족발", "치킨", "닭강정", "생선까스", "연어스테이크", "오징어볶음",
    "미나리오징어초무침", "쭈꾸미볶음", "아귀찜", "해물찜", "고등어구이", "고등어조림", "코다리조림",
    "갈치구이", "갈치조림", "장어구이", "조기구이", "가자미구이", "꽁치조림", "낙곱새", "바지락술찜",
    "바지락칼국수", "꽃게찜", "대게찜", "랍스터찜", "간장게장", "양념게장", "새우장", "회", "생굴",
    "잡채", "골뱅이무침","마늘빵", "토스트", "냉면", "잔치국수", "비빔국수", "열무국수", "콩국수", "수제비", "비빔만두", "쫄면",
    "칼국수", "떡국", "만둣국", "우동", "핫도그", "찐만두", "튀김만두", "비빔만두", "김말이튀김", "야채튀김",
    "오징어튀김", "가지튀김", "팝콘치킨", "순대", "짜장면", "짬뽕", "마라탕", "탕수육", "꿔바로우", "마파두부",
    "계란토마토볶음밥", "양장피", "깐풍기", "깐풍새우", "크림새우", "유린기", "팔보채", "고추잡채", "춘권",
    "딤섬", "토마토스파게티", "크림파스타", "명란파스타", "봉골레파스타", "감바스", "스테이크", "또띠아피자",
    "고구마그라탕", "감자그라탕", "피자", "함박스테이크", "리조또", "스테이크", "샐러드", "햄버거", "부리또",
    "해쉬브라운", "감자튀김", "맥앤치즈", "콘샐러드", "초밥", "라멘", "나가사키짬뽕", "오니기리", "연어덮밥",
    "새우장덮밥", "메밀소바", "돈카츠", "야키니쿠", "낫또", "볶음우동", "카레우동", "가츠동", "규동", "쌀국수", "팟타이", "타코"
]

# Data storage
data_food_name, data_ingredient_name_list, data_amount_list, data_cooking_time, data_nando = [], [], [], [], []

for food_name in food_list:
    food_data = food_info(food_name)
    if food_data:
        data_food_name.append(food_name)
        data_ingredient_name_list.append(food_data['ingredient_name_list'])
        data_amount_list.append(food_data['amount_list'])
        data_cooking_time.append(food_data['cooking_time'])
        data_nando.append(food_data['nando'])

# DataFrame 생성 및 CSV로 저장
food_dataframe = {
    'food_name': data_food_name,
    'ingredient_name_list': data_ingredient_name_list,
    'amount_list': data_amount_list,
    'cooking_time': data_cooking_time,
    'nando': data_nando
}

df = pd.DataFrame(food_dataframe)
df.to_csv('food_dataframe.csv', index=False)
print("food_dataframe.csv가 만들어졌습니다.")


food_dataframe.csv가 만들어졌습니다.


In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import re

def separate_amount_ingredient(input_string):
    # 정규 표현식을 사용하여 재료와 양을 나눔
    ingredient_amount_pairs = re.findall(r'([^\d]+)(\d+[\d./½¼⅓⅔⅕⅖⅗⅘⅙⅚⅛⅜⅝⅞]+)?', input_string)

    ingredient_name_list, amount_list, unit_list = [], [], []

    for pair in ingredient_amount_pairs:
        ingredient_name = pair[0].strip()
        amount_raw = pair[1].strip() if pair[1] else "None"

        # 숫자가 끝나는 지점을 찾아서 양과 단위로 분리
        if amount_raw:
            match = re.match(r'(\d+[\d./½¼⅓⅔⅕⅖⅗⅘⅙⅚⅛⅜⅝⅞]+)(.*)', amount_raw)
            if match:
                amount, unit = match.groups()
                amount_list.append(amount)
                unit_list.append(unit.strip())
            else:
                amount_list.append("None")
                unit_list.append(amount_raw)
        else:
            amount_list.append("None")
            unit_list.append("None")

        ingredient_name_list.append(ingredient_name)

    return ingredient_name_list, amount_list, unit_list

# www.10000recipe.com에서 음식 정보를 가져오는 함수
def food_info(name):
    url = f"https://www.10000recipe.com/recipe/list.html?q={name}"
    response = requests.get(url)

    if response.status_code != 200:
        print("HTTP 응답 오류:", response.status_code)
        return

    soup = BeautifulSoup(response.text, 'html.parser')
    food_list = soup.find_all(attrs={'class': 'common_sp_link'})

    if not food_list:
        print(f"'{name}'에 대한 레시피를 찾을 수 없습니다.")
        return

    food_id = food_list[0]['href'].split('/')[-1]
    new_url = f'https://www.10000recipe.com/recipe/{food_id}'
    new_response = requests.get(new_url)

    if new_response.status_code != 200:
        print("HTTP 응답 오류:", new_response.status_code)
        return

    soup = BeautifulSoup(new_response.text, 'html.parser')
    food_info = soup.find(attrs={'type': 'application/ld+json'})

    if not food_info:
        print(f"'{name}'에 대한 레시피 정보를 찾을 수 없습니다.")
        return

    result = json.loads(food_info.text)

    ingredient = ','.join(result.get('recipeIngredient', []))
    ingredient_name_list, amount_list, unit_list = separate_amount_ingredient(ingredient)

    recipe = [f'{i + 1}. {step["text"]}' for i, step in enumerate(result.get('recipeInstructions', []))]
    if not recipe:
        recipe = ["NULL"]

    cooking_time_tag = soup.find('span', {'class': 'view2_summary_info2'})
    cooking_time = cooking_time_tag.get_text(strip=True) if cooking_time_tag else "None"

    # 이미지 URL 추출
    menu_img_tag = soup.find('div', 'centeredcrop')
    menu_img_tag = menu_img_tag.find('img') if menu_img_tag else "None"
    menu_img = menu_img_tag.get('src') if menu_img_tag else "None"

    # 메뉴 정보 추출
    menu_info_1_tag = soup.find('span', 'view2_summary_info1')  # menu_info_1
    menu_info_1 = menu_info_1_tag.get_text() if menu_info_1_tag else "None"

    menu_info_3_tag = soup.find('span', 'view2_summary_info3')  # menu_info_3
    menu_info_3 = menu_info_3_tag.get_text() if menu_info_3_tag else "None"

    return {
        'name': name,
        'ingredient': ingredient_name_list,
        'amount': amount_list,
        'unit': unit_list,  # unit_list 추가
        'recipe': recipe,
        'cooking_time': cooking_time,
        'menu_img': menu_img,
        'people': menu_info_1,  # people 인덱스 추가 => 몇 인분
        'nando': menu_info_3   # nando 인덱스 추가 => 난이도
    }

# Food list
food_list = [
    "잡곡밥", "옥수수밥", "감자밥", "무밥", "명란버터밥", "비빔밥", "가지밥", "전복밥", "콩나물밥", "곤드레비빔밥"
    ]

# Data storage
data_food_name, data_food_ingredient, data_food_amount, data_food_unit, data_food_recipe, data_food_time, data_food_img, data_food_people, data_food_nando = [], [], [], [], [], [], [], [], []

for food_name in food_list:
    food_data = food_info(food_name)
    if food_data:
        data_food_name.append(food_name)
        data_food_ingredient.append(food_data['ingredient'])
        data_food_amount.append(food_data['amount'])
        data_food_unit.append(food_data['unit'])  # unit_list 추가
        data_food_recipe.append(food_data['recipe'])
        data_food_time.append(food_data['cooking_time'])
        data_food_img.append(food_data['menu_img'])
        data_food_people.append(food_data['people'])

        # 난이도 크롤링 후 숫자로 변환
        if food_data['nando'] == "아무나":
            food_data['nando'] = 1
        elif food_data['nando'] == "초급":
            food_data['nando'] = 2
        elif food_data['nando'] == "중급":
            food_data['nando'] = 3
        elif food_data['nando'] == "고급":
            food_data['nando'] = 3
        elif food_data['nando'] == "신의경지":
            food_data['nando'] = 3

        data_food_nando.append(food_data['nando'])

# DataFrame 생성 및 CSV로 저장
food_dataframe = {
    'food_name': data_food_name,
    'food_ingredient': data_food_ingredient,
    'food_amount': data_food_amount,
    'food_unit': data_food_unit,  # unit_list 추가
    'food_recipe': data_food_recipe,
    'food_time': data_food_time,
    'menu_img': data_food_img,
    'people': data_food_people,  # people 인덱스를 DataFrame에 추가
    'nando': data_food_nando,  # nando 인덱스를 DataFrame에 추가
}

df = pd.DataFrame(food_dataframe)

# 레시피 인덱스 추가
recipe_columns = [f'recipe{i + 1:02d}' for i in range(df['food_recipe'].apply(len).max())]
df[recipe_columns] = pd.DataFrame(df['food_recipe'].tolist(), index=df.index).replace('', "None")

# 필요 없는 열 삭제
df.drop(['food_recipe'], axis=1, inplace=True)

df.to_csv('food_dataframe777.csv', index=False)  # 파일명 수정
print("food_dataframe777.csv가 만들어졌습니다.")


food_dataframe777.csv가 만들어졌습니다.


In [2]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import re

def separate_amount_ingredient(input_string):
    # 정규 표현식을 사용하여 재료와 양을 나눔
    ingredient_amount_pairs = re.findall(r'(\D+)(\d+[\d./½¼⅓⅔⅕⅖⅗⅘⅙⅚⅛⅜⅝⅞]+)?(.+)?', input_string)

    ingredient_name_list, amount_list, unit_list = [], [], []

    for pair in ingredient_amount_pairs:
        ingredient_name = pair[0].strip()
        amount = pair[1].strip() if pair[1] else "None"
        unit = pair[2].strip() if pair[2] else "None"

        ingredient_name_list.append(ingredient_name)
        amount_list.append(amount)
        unit_list.append(unit)

    return ingredient_name_list, amount_list, unit_list

# www.10000recipe.com에서 음식 정보를 가져오는 함수
def food_info(name):
    url = f"https://www.10000recipe.com/recipe/list.html?q={name}"
    response = requests.get(url)

    if response.status_code != 200:
        print("HTTP 응답 오류:", response.status_code)
        return

    soup = BeautifulSoup(response.text, 'html.parser')
    food_list = soup.find_all(attrs={'class': 'common_sp_link'})

    if not food_list:
        print(f"'{name}'에 대한 레시피를 찾을 수 없습니다.")
        return

    food_id = food_list[0]['href'].split('/')[-1]
    new_url = f'https://www.10000recipe.com/recipe/{food_id}'
    new_response = requests.get(new_url)

    if new_response.status_code != 200:
        print("HTTP 응답 오류:", new_response.status_code)
        return

    soup = BeautifulSoup(new_response.text, 'html.parser')
    food_info = soup.find(attrs={'type': 'application/ld+json'})

    if not food_info:
        print(f"'{name}'에 대한 레시피 정보를 찾을 수 없습니다.")
        return

    result = json.loads(food_info.text)

    ingredient = ','.join(result.get('recipeIngredient', []))
    ingredient_name_list, amount_list, unit_list = separate_amount_ingredient(ingredient)

    recipe = [f'{i + 1}. {step["text"]}' for i, step in enumerate(result.get('recipeInstructions', []))]
    if not recipe:
        recipe = ["NULL"]

    cooking_time_tag = soup.find('span', {'class': 'view2_summary_info2'})
    cooking_time = cooking_time_tag.get_text(strip=True) if cooking_time_tag else "None"

    # 이미지 URL 추출
    menu_img_tag = soup.find('div', 'centeredcrop')
    menu_img_tag = menu_img_tag.find('img') if menu_img_tag else "None"
    menu_img = menu_img_tag.get('src') if menu_img_tag else "None"

    # 메뉴 정보 추출
    menu_info_1_tag = soup.find('span', 'view2_summary_info1')  # menu_info_1
    menu_info_1 = menu_info_1_tag.get_text() if menu_info_1_tag else "None"

    menu_info_3_tag = soup.find('span', 'view2_summary_info3')  # menu_info_3
    menu_info_3 = menu_info_3_tag.get_text() if menu_info_3_tag else "None"

    return {
        'name': name,
        'ingredient': ingredient_name_list,
        'amount': amount_list,
        'unit': unit_list,  # unit_list 추가
        'recipe': recipe,
        'cooking_time': cooking_time,
        'menu_img': menu_img,
        'people': menu_info_1,  # people 인덱스 추가 => 몇 인분
        'nando': menu_info_3   # nando 인덱스 추가 => 난이도
    }

# Food list
food_list = [
    "잡곡밥", "옥수수밥", "감자밥", "무밥", "명란버터밥", "비빔밥", "가지밥", "전복밥", "콩나물밥", "곤드레비빔밥"
    ]

# Data storage
data_food_name, data_food_ingredient, data_food_amount, data_food_unit, data_food_recipe, data_food_time, data_food_img, data_food_people, data_food_nando = [], [], [], [], [], [], [], [], []

for food_name in food_list:
    food_data = food_info(food_name)
    if food_data:
        data_food_name.append(food_name)
        data_food_ingredient.append(food_data['ingredient'])
        data_food_amount.append(food_data['amount'])
        data_food_unit.append(food_data['unit'])
        data_food_recipe.append(food_data['recipe'])
        data_food_time.append(food_data['cooking_time'])
        data_food_img.append(food_data['menu_img'])
        data_food_people.append(food_data['people'])

        # 난이도 크롤링 후 숫자로 변환
        if food_data['nando'] == "아무나":
            food_data['nando'] = 1
        elif food_data['nando'] == "초급":
            food_data['nando'] = 2
        elif food_data['nando'] == "중급":
            food_data['nando'] = 3
        elif food_data['nando'] == "고급":
            food_data['nando'] = 3
        elif food_data['nando'] == "신의경지":
            food_data['nando'] = 3

        data_food_nando.append(food_data['nando'])

# DataFrame 생성 및 CSV로 저장
food_dataframe = {
    'food_name': data_food_name,
    'food_ingredient': data_food_ingredient,
    'food_amount': data_food_amount,
    'food_unit': data_food_unit,
    'food_recipe': data_food_recipe,
    'food_time': data_food_time,
    'menu_img': data_food_img,
    'people': data_food_people,
    'nando': data_food_nando,
}

df = pd.DataFrame(food_dataframe)

# 레시피 인덱스 추가
recipe_columns = [f'recipe{i + 1:02d}' for i in range(df['food_recipe'].apply(len).max())]
df[recipe_columns] = pd.DataFrame(df['food_recipe'].tolist(), index=df.index).replace('', "None")

# 필요 없는 열 삭제
df.drop(['food_recipe'], axis=1, inplace=True)

df.to_csv('food_dataframe777.csv', index=False)
print("food_dataframe777.csv가 만들어졌습니다.")


food_dataframe777.csv가 만들어졌습니다.


In [6]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

# www.10000recipe.com에서 음식 정보를 가져오는 함수
def food_info(name):
    url = f"https://www.10000recipe.com/recipe/list.html?q={name}"
    response = requests.get(url)

    if response.status_code != 200:
        print("HTTP 응답 오류:", response.status_code)
        return

    soup = BeautifulSoup(response.text, 'html.parser')
    food_list = soup.find_all(attrs={'class': 'common_sp_link'})

    if not food_list:
        print(f"'{name}'에 대한 레시피를 찾을 수 없습니다.")
        return

    food_id = food_list[0]['href'].split('/')[-1]
    new_url = f'https://www.10000recipe.com/recipe/{food_id}'
    new_response = requests.get(new_url)

    if new_response.status_code != 200:
        print("HTTP 응답 오류:", new_response.status_code)
        return

    soup = BeautifulSoup(new_response.text, 'html.parser')
    food_info = soup.find(attrs={'type': 'application/ld+json'})

    if not food_info:
        print(f"'{name}'에 대한 레시피 정보를 찾을 수 없습니다.")
        return

    result = json.loads(food_info.text)

    ingredients = result.get('recipeIngredient', [])
    ingredient_list = [ingredient.strip() for ingredient in ingredients]

    recipe = [f'{i + 1}. {step["text"]}' for i, step in enumerate(result.get('recipeInstructions', []))]

    cooking_time_tag = soup.find('span', {'class': 'view2_summary_info2'})
    cooking_time = cooking_time_tag.get_text(strip=True) if cooking_time_tag else "None"

    # 이미지 URL 추출
    menu_img_tag = soup.find('div', 'centeredcrop')
    menu_img_tag = menu_img_tag.find('img') if menu_img_tag else "None"
    menu_img = menu_img_tag.get('src') if menu_img_tag else "None"

    # 메뉴 정보 추출
    menu_info_1_tag = soup.find('span', 'view2_summary_info1')  # menu_info_1
    menu_info_1 = menu_info_1_tag.get_text() if menu_info_1_tag else "None"

    menu_info_3_tag = soup.find('span', 'view2_summary_info3')  # menu_info_3
    menu_info_3 = menu_info_3_tag.get_text() if menu_info_3_tag else "None"

    return {
        'name': name,
        'ingredient': ingredient_list,
        'recipe': recipe,
        'cooking_time': cooking_time,
        'menu_img': menu_img,
        'people': menu_info_1,  # people 인덱스 추가 => 몇 인분
        'nando': menu_info_3   # nando 인덱스 추가 => 난이도
    }

# Food list
food_list = [
    "딤섬", "토마토스파게티", "크림파스타"]

# Data storage
data_food_name, data_food_ingredient, data_food_recipe, data_food_time, data_food_img, data_food_people, data_food_nando = [], [], [], [], [], [], []

for food_name in food_list:
    food_data = food_info(food_name)
    if food_data:
        data_food_name.append(food_name)
        data_food_ingredient.append(food_data['ingredient'])
        data_food_recipe.append(food_data['recipe'])
        data_food_time.append(food_data['cooking_time'])
        data_food_img.append(food_data['menu_img'])
        data_food_people.append(food_data['people'])

        # 난이도 크롤링 후 숫자로 변환
        if food_data['nando'] == "아무나" :
            food_data['nando'] = 1
        elif food_data['nando'] == "초급" :
            food_data['nando'] = 2
        elif food_data['nando'] == "중급" :
            food_data['nando'] = 3
        elif food_data['nando'] == "고급" :
            food_data['nando'] = 3
        elif food_data['nando'] == "신의경지":
            food_data['nando'] = 3

        data_food_nando.append(food_data['nando'])


# DataFrame 생성 및 CSV로 저장
food_dataframe = {
    'food_name': data_food_name,
    'food_ingredient': data_food_ingredient,
    'food_recipe': data_food_recipe,
    'food_time': data_food_time,
    'menu_img': data_food_img,
    'people': data_food_people,  # people 인덱스를 DataFrame에 추가
    'nando': data_food_nando,  # nando 인덱스를 DataFrame에 추가
}

df = pd.DataFrame(food_dataframe)
df.to_csv('food_dataframe14.csv', index=False)
print("food_dataframe14.csv가 만들어졌습니다.")


food_dataframe14.csv가 만들어졌습니다.


In [7]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import re

# www.10000recipe.com에서 음식 정보를 가져오는 함수
def food_info(name):
    url = f"https://www.10000recipe.com/recipe/list.html?q={name}"
    response = requests.get(url)

    if response.status_code != 200:
        print("HTTP 응답 오류:", response.status_code)
        return

    soup = BeautifulSoup(response.text, 'html.parser')
    food_list = soup.find_all(attrs={'class': 'common_sp_link'})

    if not food_list:
        print(f"'{name}'에 대한 레시피를 찾을 수 없습니다.")
        return

    food_id = food_list[0]['href'].split('/')[-1]
    new_url = f'https://www.10000recipe.com/recipe/{food_id}'
    new_response = requests.get(new_url)

    if new_response.status_code != 200:
        print("HTTP 응답 오류:", new_response.status_code)
        return

    soup = BeautifulSoup(new_response.text, 'html.parser')
    food_info = soup.find(attrs={'type': 'application/ld+json'})

    if not food_info:
        print(f"'{name}'에 대한 레시피 정보를 찾을 수 없습니다.")
        return

    result = json.loads(food_info.text)

    ingredients = result.get('recipeIngredient', [])
    ingredient_name_list, ingredient_amount_list, ingredient_unit_list = [], [], []

    for ingredient in ingredients:
        match = re.match(r'(\d+(\.\d+)?)\s*([^\d]+)', ingredient)
        if match:
            ingredient_amount_list.append(match.group(1))
            ingredient_name_list.append(match.group(3).strip())
            ingredient_unit_list.append(match.group(3).replace(match.group(3).strip(), '').strip())
        else:
            ingredient_amount_list.append("None")
            ingredient_name_list.append(ingredient.strip())
            ingredient_unit_list.append("None")

    recipe = [f'{i + 1}. {step["text"]}' for i, step in enumerate(result.get('recipeInstructions', []))]

    cooking_time_tag = soup.find('span', {'class': 'view2_summary_info2'})
    cooking_time = cooking_time_tag.get_text(strip=True) if cooking_time_tag else "None"

    # 이미지 URL 추출
    menu_img_tag = soup.find('div', 'centeredcrop')
    menu_img_tag = menu_img_tag.find('img') if menu_img_tag else "None"
    menu_img = menu_img_tag.get('src') if menu_img_tag else "None"

    # 메뉴 정보 추출
    menu_info_1_tag = soup.find('span', 'view2_summary_info1')  # menu_info_1
    menu_info_1 = menu_info_1_tag.get_text() if menu_info_1_tag else "None"

    menu_info_3_tag = soup.find('span', 'view2_summary_info3')  # menu_info_3
    menu_info_3 = menu_info_3_tag.get_text() if menu_info_3_tag else "None"

    return {
        'name': name,
        'ingredient_name': ingredient_name_list,
        'ingredient_amount': ingredient_amount_list,
        'ingredient_unit': ingredient_unit_list,
        'recipe': recipe,
        'cooking_time': cooking_time,
        'menu_img': menu_img,
        'people': menu_info_1,  # people 인덱스 추가 => 몇 인분
        'nando': menu_info_3   # nando 인덱스 추가 => 난이도
    }

# Food list
food_list = [
    "딤섬", "토마토스파게티", "크림파스타"]

# Data storage
data_food_name, data_food_ingredient_name, data_food_ingredient_amount, data_food_ingredient_unit, data_food_recipe, data_food_time, data_food_img, data_food_people, data_food_nando = [], [], [], [], [], [], [], [], []

for food_name in food_list:
    food_data = food_info(food_name)
    if food_data:
        data_food_name.append(food_name)
        data_food_ingredient_name.append(food_data['ingredient_name'])
        data_food_ingredient_amount.append(food_data['ingredient_amount'])
        data_food_ingredient_unit.append(food_data['ingredient_unit'])
        data_food_recipe.append(food_data['recipe'])
        data_food_time.append(food_data['cooking_time'])
        data_food_img.append(food_data['menu_img'])
        data_food_people.append(food_data['people'])

        # 난이도 크롤링 후 숫자로 변환
        if food_data['nando'] == "아무나" :
            food_data['nando'] = 1
        elif food_data['nando'] == "초급" :
            food_data['nando'] = 2
        elif food_data['nando'] == "중급" :
            food_data['nando'] = 3
        elif food_data['nando'] == "고급" :
            food_data['nando'] = 3
        elif food_data['nando'] == "신의경지":
            food_data['nando'] = 3

        data_food_nando.append(food_data['nando'])


# DataFrame 생성 및 CSV로 저장
food_dataframe = {
    'food_name': data_food_name,
    'ingredient_name': data_food_ingredient_name,
    'ingredient_amount': data_food_ingredient_amount,
    'ingredient_unit': data_food_ingredient_unit,
    'food_recipe': data_food_recipe,
    'food_time': data_food_time,
    'menu_img': data_food_img,
    'people': data_food_people,  # people 인덱스를 DataFrame에 추가
    'nando': data_food_nando,  # nando 인덱스를 DataFrame에 추가
}

df = pd.DataFrame(food_dataframe)
df.to_csv('food_dataframe15.csv', index=False)
print("food_dataframe15.csv가 만들어졌습니다.")


food_dataframe15.csv가 만들어졌습니다.


In [9]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import re
from fractions import Fraction

# www.10000recipe.com에서 음식 정보를 가져오는 함수
def food_info(name):
    url = f"https://www.10000recipe.com/recipe/list.html?q={name}"
    response = requests.get(url)

    if response.status_code != 200:
        print("HTTP 응답 오류:", response.status_code)
        return

    soup = BeautifulSoup(response.text, 'html.parser')
    food_list = soup.find_all(attrs={'class': 'common_sp_link'})

    if not food_list:
        print(f"'{name}'에 대한 레시피를 찾을 수 없습니다.")
        return

    food_id = food_list[0]['href'].split('/')[-1]
    new_url = f'https://www.10000recipe.com/recipe/{food_id}'
    new_response = requests.get(new_url)

    if new_response.status_code != 200:
        print("HTTP 응답 오류:", new_response.status_code)
        return

    soup = BeautifulSoup(new_response.text, 'html.parser')
    food_info = soup.find(attrs={'type': 'application/ld+json'})

    if not food_info:
        print(f"'{name}'에 대한 레시피 정보를 찾을 수 없습니다.")
        return

    result = json.loads(food_info.text)

    ingredients = result.get('recipeIngredient', [])
    ingredient_name_list, ingredient_amount_list, ingredient_unit_list = [], [], []

    for ingredient in ingredients:
        match = re.match(r'(\d+(\.\d+)?(/\d+(\.\d+)?)?)\s*(\S+)', ingredient)
        if match:
            fraction_str = match.group(1)
            ingredient_amount = float(Fraction(fraction_str))
            ingredient_amount_list.append(ingredient_amount)
            ingredient_name_list.append(match.group(5).strip())
            ingredient_unit_list.append(match.group(5).replace(match.group(5).strip(), '').strip())
        else:
            ingredient_amount_list.append("None")
            ingredient_name_list.append(ingredient.strip())
            ingredient_unit_list.append("None")

    recipe = [f'{i + 1}. {step["text"]}' for i, step in enumerate(result.get('recipeInstructions', []))]

    cooking_time_tag = soup.find('span', {'class': 'view2_summary_info2'})
    cooking_time = cooking_time_tag.get_text(strip=True) if cooking_time_tag else "None"

    # 이미지 URL 추출
    menu_img_tag = soup.find('div', 'centeredcrop')
    menu_img_tag = menu_img_tag.find('img') if menu_img_tag else "None"
    menu_img = menu_img_tag.get('src') if menu_img_tag else "None"

    # 메뉴 정보 추출
    menu_info_1_tag = soup.find('span', 'view2_summary_info1')  # menu_info_1
    menu_info_1 = menu_info_1_tag.get_text() if menu_info_1_tag else "None"

    menu_info_3_tag = soup.find('span', 'view2_summary_info3')  # menu_info_3
    menu_info_3 = menu_info_3_tag.get_text() if menu_info_3_tag else "None"

    return {
        'name': name,
        'ingredient_name': ingredient_name_list,
        'ingredient_amount': ingredient_amount_list,
        'ingredient_unit': ingredient_unit_list,
        'recipe': recipe,
        'cooking_time': cooking_time,
        'menu_img': menu_img,
        'people': menu_info_1,  # people 인덱스 추가 => 몇 인분
        'nando': menu_info_3   # nando 인덱스 추가 => 난이도
    }

# Food list
food_list = [
    "딤섬", "토마토스파게티", "크림파스타"]

# Data storage
data_food_name, data_food_ingredient_name, data_food_ingredient_amount, data_food_ingredient_unit, data_food_recipe, data_food_time, data_food_img, data_food_people, data_food_nando = [], [], [], [], [], [], [], [], []

for food_name in food_list:
    food_data = food_info(food_name)
    if food_data:
        data_food_name.append(food_name)
        data_food_ingredient_name.append(food_data['ingredient_name'])
        data_food_ingredient_amount.append(food_data['ingredient_amount'])
        data_food_ingredient_unit.append(food_data['ingredient_unit'])
        data_food_recipe.append(food_data['recipe'])
        data_food_time.append(food_data['cooking_time'])
        data_food_img.append(food_data['menu_img'])
        data_food_people.append(food_data['people'])

        # 난이도 크롤링 후 숫자로 변환
        if food_data['nando'] == "아무나" :
            food_data['nando'] = 1
        elif food_data['nando'] == "초급" :
            food_data['nando'] = 2
        elif food_data['nando'] == "중급" :
            food_data['nando'] = 3
        elif food_data['nando'] == "고급" :
            food_data['nando'] = 3
        elif food_data['nando'] == "신의경지":
            food_data['nando'] = 3

        data_food_nando.append(food_data['nando'])

# DataFrame 생성 및 CSV로 저장
food_dataframe = {
    'food_name': data_food_name,
    'ingredient_name': data_food_ingredient_name,
    'ingredient_amount': data_food_ingredient_amount,
    'ingredient_unit': data_food_ingredient_unit,
    'food_recipe': data_food_recipe,
    'food_time': data_food_time,
    'menu_img': data_food_img,
    'people': data_food_people,  # people 인덱스를 DataFrame에 추가
    'nando': data_food_nando,  # nando 인덱스를 DataFrame에 추가
}

df = pd.DataFrame(food_dataframe)
df.to_csv('food_dataframe14.csv', index=False)
print("food_dataframe14.csv가 만들어졌습니다.")


food_dataframe14.csv가 만들어졌습니다.


In [ ]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv('/content/drive/MyDrive/wimf/LAST_food_dataframe11.csv', encoding='cp949')

# 'recipe01'부터 'recipe25'까지의 컬럼을 리스트로 병합하여 새로운 컬럼 생성
recipe_columns = [f'recipe{i:d}' for i in range(1, 26)]
df['merged_recipe'] = df.apply(lambda row: row[recipe_columns].tolist(), axis=1)

# 결과를 확인하고 필요에 따라 적절히 수정
print(df)

# 수정된 데이터프레임을 새로운 CSV 파일로 저장
df.to_csv('/content/drive/MyDrive/wimf/LAST_MODI_food_dataframe11.csv', index=False, encoding='utf-8')
print("modified_csv_file.csv가 만들어졌습니다.")


In [11]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv('/content/drive/MyDrive/wimf/LAST_MODI_food_dataframe11.csv', encoding='utf-8')

# 'merged_recipe' 컬럼에서 None 값을 제거하여 새로운 컬럼 생성
df['filtered_merged_recipe'] = df['merged_recipe'].apply(lambda x: [value for value in x if pd.notna(value)])

# 필요에 따라 적절히 수정
print(df)

# 수정된 데이터프레임을 새로운 CSV 파일로 저장
df.to_csv('/content/drive/MyDrive/wimf/LAST_MODI_food_dataframe11_filtered.csv', index=False, encoding='utf-8')
print("filtered_csv_file.csv가 만들어졌습니다.")


      title                                         ingredient  \
0      LA갈비  ['LA갈비', '간장', '설탕', '물엿', '마늘', '대파', '양파', '...   
1     가자미구이  ['손질한냉동가자미', '전분', '파채', '라임(레몬)', '쪽파', '물', ...   
2       가지밥  ['현미', '가지', '파', '올리브유', '간장', '부추', '파', '마늘...   
3      가지튀김  ['가지', '식용유', '계란', '밀가루', '빵가루', '소금', '간장', ...   
4       가츠동  ['돈가스', '밥', '양파', '계란', '다시마', '간장', '맛술', '설...   
..      ...                                                ...   
208   해쉬브라운               ['삶은감자', '생감자', '전분가루', '맛소금', '후추']   
209     햄버거  ['돼지고기간것', '소고기간것', '양파', '피클', '파프리카', '치즈', ...   
210     홍합탕             ['홍합', '대파', '양파', '청양고추', '다진', '소금']   
211       회  ['연어', '새싹채소', '밥', '고추장', '식초', '매실액', '올리고당'...   
212  훈제오리구이  ['훈제오리', '부추', '양파', '팽이버섯', '고추장', '고추가루', '다...   

                                            food_amout  \
0           [1, 10, 2, 4, 1, 10, 1/2, 1/4, 1, None, 1]   
1    [2, None, None, None, 4, 3, 2, 1, 1, 1, 3, Non...   
2    [2, 2, 1, 4, 3, 1/2, 1/2, 1/

In [16]:
## 음식 소개만 따오도록
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

def get_recipe_intro(name):
    url = f"https://www.10000recipe.com/recipe/list.html?q={name}"
    response = requests.get(url)

    if response.status_code != 200:
        print("HTTP 응답 오류:", response.status_code)
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    food_list = soup.find_all(attrs={'class': 'common_sp_link'})

    if not food_list:
        print(f"'{name}'에 대한 레시피를 찾을 수 없습니다.")
        return None

    food_id = food_list[0]['href'].split('/')[-1]
    new_url = f'https://www.10000recipe.com/recipe/{food_id}'
    new_response = requests.get(new_url)

    if new_response.status_code != 200:
        print("HTTP 응답 오류:", new_response.status_code)
        return None

    soup = BeautifulSoup(new_response.text, 'html.parser')
    food_info = soup.find(attrs={'type': 'application/ld+json'})

    if not food_info:
        print(f"'{name}'에 대한 레시피 정보를 찾을 수 없습니다.")
        return None

    result = json.loads(food_info.text)

    # 음식의 이름과 소개만 추출
    food_name = result.get('name', 'None')
    recipe_intro = result.get('description', 'None')

    return {
        'food_name': food_name,
        'recipe_intro': recipe_intro
    }

# Food list
food_list = ["잡곡밥", "옥수수밥", "감자밥", "무밥", "명란버터밥", "비빔밥", "가지밥", "전복밥", "콩나물밥", "곤드레비빔밥",
    "표고버섯영양밥", "쌈밥", "야채죽", "전복죽", "새우죽", "삼계죽", "미역죽", "참치죽", "소고기버섯죽", "팥죽",
    "단호박죽", "베이컨볶음밥", "김치볶음밥", "간장계란밥", "소고기볶음밥", "스팸볶음밥", "해물볶음밥", "새우볶음밥",
    "카레덮밥", "짜장밥", "오징어덮밥", "오므라이스", "육회비빔밥", "김치알밥", "미역국", "무국", "콩나물국",
    "김치콩나물국", "사골곰탕", "북엇국", "우거지국", "시래기국", "배추된장국", "매생이국", "올갱이국", "뼈해장국",
    "된장국", "계란감자국", "계란국", "감자국", "오징어무국", "어묵탕", "육개장", "갈비탕", "삼계탕", "추어탕",
    "꽃게탕", "홍합탕", "해물누룽지탕", "된장찌개", "차돌된장찌개", "꽃게된장찌개", "김치찌개", "순두부찌개",
    "부대찌개", "청국장", "동태찌개", "비지찌개", "고추장찌개", "짜글이찌개", "버섯찌개", "소고기찌개",
    "새우찌개", "밀폐유나베", "소고기버섯전골", "불고기전골", "어묵전골", "만두전골", "두부전골", "버섯전골",
    "곱창전골", "대창전골", "아롱사태전골", "불낙전골", "삼겹살", "수육", "스테이크구이", "찹스테이크",
    "갈비찜", "돼지갈비", "LA갈비", "바베큐", "김치등갈비찜", "묵은지돼지갈비찜", "매운쪽갈비찜", "폭립",
    "제육볶음","소불고기", "돼지불고기", "닭볶음탕", "닭갈비", "훈제오리구이", "단호박훈제오리찜", "삼겹살숙주볶음",
    "차돌숙주볶음", "찜닭", "소세지야채볶음", "돈까스", "떡갈비", "함박스테이크", "동그랑땡", "편백나무찜",
    "곱창구이", "막창구이", "족발", "치킨", "닭강정", "생선까스", "연어스테이크", "오징어볶음",
    "미나리오징어초무침", "쭈꾸미볶음", "아귀찜", "해물찜", "고등어구이", "고등어조림", "코다리조림",
    "갈치구이", "갈치조림", "장어구이", "조기구이", "가자미구이", "꽁치조림", "낙곱새", "바지락술찜",
    "바지락칼국수", "꽃게찜", "대게찜", "랍스터찜", "간장게장", "양념게장", "새우장", "회", "생굴",
    "잡채", "골뱅이무침","마늘빵", "토스트", "냉면", "잔치국수", "비빔국수", "열무국수", "콩국수", "수제비", "비빔만두", "쫄면",
    "칼국수", "떡국", "만둣국", "우동", "핫도그", "찐만두", "튀김만두", "비빔만두", "김말이튀김", "야채튀김",
    "오징어튀김", "가지튀김", "팝콘치킨", "순대", "짜장면", "짬뽕", "마라탕", "탕수육", "꿔바로우", "마파두부",
    "계란토마토볶음밥", "양장피", "깐풍기", "깐풍새우", "크림새우", "유린기", "팔보채", "고추잡채", "춘권",
    "딤섬", "토마토스파게티", "크림파스타", "명란파스타", "봉골레파스타", "감바스", "스테이크", "또띠아피자",
    "고구마그라탕", "감자그라탕", "피자", "함박스테이크", "리조또", "스테이크", "샐러드", "햄버거", "부리또",
    "해쉬브라운", "감자튀김", "맥앤치즈", "콘샐러드", "초밥", "라멘", "나가사키짬뽕", "오니기리", "연어덮밥",
    "새우장덮밥", "메밀소바", "돈카츠", "야키니쿠", "낫또", "볶음우동", "카레우동", "가츠동", "규동", "쌀국수", "팟타이", "타코"]

# Data storage
data_food_name, data_recipe_intro = [], []

for food_name in food_list:
    recipe_data = get_recipe_intro(food_name)
    if recipe_data:
        data_food_name.append(recipe_data['food_name'])
        data_recipe_intro.append(recipe_data['recipe_intro'])

# DataFrame 생성 및 CSV로 저장
recipe_intro_dataframe = {
    'food_name': data_food_name,
    'recipe_intro': data_recipe_intro
}

df = pd.DataFrame(recipe_intro_dataframe)
df.to_csv('recipe_intro_dataframe.csv', index=False)
print("recipe_intro_dataframe.csv가 만들어졌습니다.")


recipe_intro_dataframe.csv가 만들어졌습니다.


In [17]:
## 음식 소개만 따오도록
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

def get_recipe_intro(search_query):
    url = f"https://www.10000recipe.com/recipe/list.html?q={search_query}"
    response = requests.get(url)

    if response.status_code != 200:
        print("HTTP 응답 오류:", response.status_code)
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    food_list = soup.find_all(attrs={'class': 'common_sp_link'})

    if not food_list:
        print(f"'{search_query}'에 대한 레시피를 찾을 수 없습니다.")
        return None

    food_id = food_list[0]['href'].split('/')[-1]
    new_url = f'https://www.10000recipe.com/recipe/{food_id}'
    new_response = requests.get(new_url)

    if new_response.status_code != 200:
        print("HTTP 응답 오류:", new_response.status_code)
        return None

    soup = BeautifulSoup(new_response.text, 'html.parser')
    food_info = soup.find(attrs={'type': 'application/ld+json'})

    if not food_info:
        print(f"'{search_query}'에 대한 레시피 정보를 찾을 수 없습니다.")
        return None

    result = json.loads(food_info.text)

    # 음식의 이름과 소개만 추출
    food_name = result.get('name', 'None')
    recipe_intro = result.get('description', 'None')

    return {
        'food_name': food_name,
        'recipe_intro': recipe_intro
    }

# 검색어 리스트
search_queries = ["잡곡밥", "옥수수밥", "감자밥", "무밥", "명란버터밥", "비빔밥", "가지밥", "전복밥", "콩나물밥", "곤드레비빔밥",
    "표고버섯영양밥", "쌈밥", "야채죽", "전복죽", "새우죽", "삼계죽", "미역죽", "참치죽", "소고기버섯죽", "팥죽",
    "단호박죽", "베이컨볶음밥", "김치볶음밥", "간장계란밥", "소고기볶음밥", "스팸볶음밥", "해물볶음밥", "새우볶음밥",
    "카레덮밥", "짜장밥", "오징어덮밥", "오므라이스", "육회비빔밥", "김치알밥", "미역국", "무국", "콩나물국",
    "김치콩나물국", "사골곰탕", "북엇국", "우거지국", "시래기국", "배추된장국", "매생이국", "올갱이국", "뼈해장국",
    "된장국", "계란감자국", "계란국", "감자국", "오징어무국", "어묵탕", "육개장", "갈비탕", "삼계탕", "추어탕",
    "꽃게탕", "홍합탕", "해물누룽지탕", "된장찌개", "차돌된장찌개", "꽃게된장찌개", "김치찌개", "순두부찌개",
    "부대찌개", "청국장", "동태찌개", "비지찌개", "고추장찌개", "짜글이찌개", "버섯찌개", "소고기찌개",
    "새우찌개", "밀폐유나베", "소고기버섯전골", "불고기전골", "어묵전골", "만두전골", "두부전골", "버섯전골",
    "곱창전골", "대창전골", "아롱사태전골", "불낙전골", "삼겹살", "수육", "스테이크구이", "찹스테이크",
    "갈비찜", "돼지갈비", "LA갈비", "바베큐", "김치등갈비찜", "묵은지돼지갈비찜", "매운쪽갈비찜", "폭립",
    "제육볶음","소불고기", "돼지불고기", "닭볶음탕", "닭갈비", "훈제오리구이", "단호박훈제오리찜", "삼겹살숙주볶음",
    "차돌숙주볶음", "찜닭", "소세지야채볶음", "돈까스", "떡갈비", "함박스테이크", "동그랑땡", "편백나무찜",
    "곱창구이", "막창구이", "족발", "치킨", "닭강정", "생선까스", "연어스테이크", "오징어볶음",
    "미나리오징어초무침", "쭈꾸미볶음", "아귀찜", "해물찜", "고등어구이", "고등어조림", "코다리조림",
    "갈치구이", "갈치조림", "장어구이", "조기구이", "가자미구이", "꽁치조림", "낙곱새", "바지락술찜",
    "바지락칼국수", "꽃게찜", "대게찜", "랍스터찜", "간장게장", "양념게장", "새우장", "회", "생굴",
    "잡채", "골뱅이무침","마늘빵", "토스트", "냉면", "잔치국수", "비빔국수", "열무국수", "콩국수", "수제비", "비빔만두", "쫄면",
    "칼국수", "떡국", "만둣국", "우동", "핫도그", "찐만두", "튀김만두", "비빔만두", "김말이튀김", "야채튀김",
    "오징어튀김", "가지튀김", "팝콘치킨", "순대", "짜장면", "짬뽕", "마라탕", "탕수육", "꿔바로우", "마파두부",
    "계란토마토볶음밥", "양장피", "깐풍기", "깐풍새우", "크림새우", "유린기", "팔보채", "고추잡채", "춘권",
    "딤섬", "토마토스파게티", "크림파스타", "명란파스타", "봉골레파스타", "감바스", "스테이크", "또띠아피자",
    "고구마그라탕", "감자그라탕", "피자", "함박스테이크", "리조또", "스테이크", "샐러드", "햄버거", "부리또",
    "해쉬브라운", "감자튀김", "맥앤치즈", "콘샐러드", "초밥", "라멘", "나가사키짬뽕", "오니기리", "연어덮밥",
    "새우장덮밥", "메밀소바", "돈카츠", "야키니쿠", "낫또", "볶음우동", "카레우동", "가츠동", "규동", "쌀국수", "팟타이", "타코"]

# Data storage
data_food_name, data_recipe_intro = [], []

for search_query in search_queries:
    recipe_data = get_recipe_intro(search_query)
    if recipe_data:
        data_food_name.append(recipe_data['food_name'])
        data_recipe_intro.append(recipe_data['recipe_intro'])

# DataFrame 생성 및 CSV로 저장
recipe_intro_dataframe = {
    'food_name': data_food_name,
    'recipe_intro': data_recipe_intro
}

df = pd.DataFrame(recipe_intro_dataframe)
df.to_csv('recipe_intro_dataframe.csv', index=False)
print("recipe_intro_dataframe.csv가 만들어졌습니다.")


KeyboardInterrupt: 

In [21]:
## 음식 소개만 따오도록
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

def get_recipe_intro(search_query):
    url = f"https://www.10000recipe.com/recipe/list.html?q={search_query}"
    response = requests.get(url)

    if response.status_code != 200:
        print("HTTP 응답 오류:", response.status_code)
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    food_list = soup.find_all(attrs={'class': 'common_sp_link'})

    if not food_list:
        print(f"'{search_query}'에 대한 레시피를 찾을 수 없습니다.")
        return None

    food_id = food_list[0]['href'].split('/')[-1]
    new_url = f'https://www.10000recipe.com/recipe/{food_id}'
    new_response = requests.get(new_url)

    if new_response.status_code != 200:
        print("HTTP 응답 오류:", new_response.status_code)
        return None

    soup = BeautifulSoup(new_response.text, 'html.parser')
    food_info = soup.find(attrs={'type': 'application/ld+json'})

    if not food_info:
        print(f"'{search_query}'에 대한 레시피 정보를 찾을 수 없습니다.")
        return None

    result = json.loads(food_info.text)

    # 음식의 이름과 소개만 추출
    food_name = search_query
    recipe_intro = result.get('description', 'None')

    return {
        'food_name': food_name,
        'recipe_intro': recipe_intro
    }

# 검색어 리스트
search_queries = ["잡곡밥", "옥수수밥", "감자밥", "무밥", "명란버터밥", "비빔밥", "가지밥", "전복밥", "콩나물밥", "곤드레비빔밥",
    "표고버섯영양밥", "쌈밥", "야채죽", "전복죽", "새우죽", "삼계죽", "미역죽", "참치죽", "소고기버섯죽", "팥죽",
    "단호박죽", "베이컨볶음밥", "김치볶음밥", "간장계란밥", "소고기볶음밥", "스팸볶음밥", "해물볶음밥", "새우볶음밥",
    "카레덮밥", "짜장밥", "오징어덮밥", "오므라이스", "육회비빔밥", "김치알밥", "미역국", "무국", "콩나물국",
    "김치콩나물국", "사골곰탕", "북엇국", "우거지국", "시래기국", "배추된장국", "매생이국", "올갱이국", "뼈해장국",
    "된장국", "계란감자국", "계란국", "감자국", "오징어무국", "어묵탕", "육개장", "갈비탕", "삼계탕", "추어탕",
    "꽃게탕", "홍합탕", "해물누룽지탕", "된장찌개", "차돌된장찌개", "꽃게된장찌개", "김치찌개", "순두부찌개",
    "부대찌개", "청국장", "동태찌개", "비지찌개", "고추장찌개", "짜글이찌개", "버섯찌개", "소고기찌개",
    "새우찌개", "밀폐유나베", "소고기버섯전골", "불고기전골", "어묵전골", "만두전골", "두부전골", "버섯전골",
    "곱창전골", "대창전골", "아롱사태전골", "불낙전골", "삼겹살", "수육", "스테이크구이", "찹스테이크",
    "갈비찜", "돼지갈비", "LA갈비", "바베큐", "김치등갈비찜", "묵은지돼지갈비찜", "매운쪽갈비찜", "폭립",
    "제육볶음","소불고기", "돼지불고기", "닭볶음탕", "닭갈비", "훈제오리구이", "단호박훈제오리찜", "삼겹살숙주볶음",
    "차돌숙주볶음", "찜닭", "소세지야채볶음", "돈까스", "떡갈비", "함박스테이크", "동그랑땡", "편백나무찜",
    "곱창구이", "막창구이", "족발", "치킨", "닭강정", "생선까스", "연어스테이크", "오징어볶음",
    "미나리오징어초무침", "쭈꾸미볶음", "아귀찜", "해물찜", "고등어구이", "고등어조림", "코다리조림",
    "갈치구이", "갈치조림", "장어구이", "조기구이", "가자미구이", "꽁치조림", "낙곱새", "바지락술찜",
    "바지락칼국수", "꽃게찜", "대게찜", "랍스터찜", "간장게장", "양념게장", "새우장", "회", "생굴",
    "잡채", "골뱅이무침","마늘빵", "토스트", "냉면", "잔치국수", "비빔국수", "열무국수", "콩국수", "수제비", "비빔만두", "쫄면",
    "칼국수", "떡국", "만둣국", "우동", "핫도그", "찐만두", "튀김만두", "비빔만두", "김말이튀김", "야채튀김",
    "오징어튀김", "가지튀김", "팝콘치킨", "순대", "짜장면", "짬뽕", "마라탕", "탕수육", "꿔바로우", "마파두부",
    "계란토마토볶음밥", "양장피", "깐풍기", "깐풍새우", "크림새우", "유린기", "팔보채", "고추잡채", "춘권",
    "딤섬", "토마토스파게티", "크림파스타", "명란파스타", "봉골레파스타", "감바스", "스테이크", "또띠아피자",
    "고구마그라탕", "감자그라탕", "피자", "함박스테이크", "리조또", "스테이크", "샐러드", "햄버거", "부리또",
    "해쉬브라운", "감자튀김", "맥앤치즈", "콘샐러드", "초밥", "라멘", "나가사키짬뽕", "오니기리", "연어덮밥",
    "새우장덮밥", "메밀소바", "돈카츠", "야키니쿠", "낫또", "볶음우동", "카레우동", "가츠동", "규동", "쌀국수", "팟타이", "타코"]

# Data storage
data_food_name, data_recipe_intro = [], []

for search_query in search_queries:
    recipe_data = get_recipe_intro(search_query)
    if recipe_data:
        data_food_name.append(recipe_data['food_name'])
        data_recipe_intro.append(recipe_data['recipe_intro'])

# DataFrame 생성 및 CSV로 저장
recipe_intro_dataframe = {
    'food_name': data_food_name,
    'recipe_intro': data_recipe_intro
}

df = pd.DataFrame(recipe_intro_dataframe)
df.to_csv('examaaa.csv', index=False)
print("recipe_intro_dataframe.csv가 만들어졌습니다.")


recipe_intro_dataframe.csv가 만들어졌습니다.


In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

# www.10000recipe.com에서 음식 정보를 가져오는 함수
def food_info(name):
    url = f"https://www.10000recipe.com/recipe/list.html?q={name}"
    response = requests.get(url)

    if response.status_code != 200:
        print("HTTP 응답 오류:", response.status_code)
        return

    soup = BeautifulSoup(response.text, 'html.parser')
    food_list = soup.find_all(attrs={'class': 'common_sp_link'})

    if not food_list:
  ...

In [11]:
import csv
import json

def process_ingredient(ingredient_str):
    # JSON 형식의 배열로 변환
    ingredient_list = json.loads(ingredient_str.replace("'", '"'))
    return ingredient_list

def process_food_data(data_str):
    # JSON 형식의 배열로 변환
    data_list = json.loads(data_str.replace("'", '"'))
    return data_list

def csv_to_json(input_csv_path, output_json_path, encoding='utf-8'):
    # CSV 파일 열기
    with open(input_csv_path, 'r', encoding=encoding) as csv_file:
        # CSV 파일을 읽어오는 reader 생성
        csv_reader = csv.DictReader(csv_file)

        # JSON으로 변환할 데이터를 담을 리스트 생성
        data = []

        # CSV 데이터를 읽어서 리스트에 추가
        for row in csv_reader:
            # ingredient 필드를 JSON 형식의 배열로 변환
            row['ingredient'] = process_ingredient(row['ingredient'])

            # food_amount, food_unit, steps 컬럼을 JSON 형식의 배열로 변환
            row['food_amount'] = process_food_data(row['food_amount'])
            row['food_unit'] = process_food_data(row['food_unit'])
            row['steps'] = process_food_data(row['steps'])

            data.append(row)

    # JSON 파일로 쓰기
    with open(output_json_path, 'w', encoding='utf-8') as json_file:
        # JSON 파일에 데이터 쓰기
        json.dump(data, json_file, ensure_ascii=False, indent=2)

if __name__ == "__main__":
    # 변환할 CSV 파일 경로 및 JSON 파일 경로 설정
    input_csv_path = "/content/drive/MyDrive/wimf/ss_LAST_MODI_food_dataframe11.csv"
    output_json_path = '/content/drive/MyDrive/wimf/ss_LAST_MODI_food_dataframe11.json'

    # CSV를 JSON으로 변환 (만약 다른 인코딩이라면, encoding 매개변수를 해당 인코딩으로 변경)
    csv_to_json(input_csv_path, output_json_path, encoding='cp949')


UnicodeDecodeError: 'cp949' codec can't decode byte 0x80 in position 168: illegal multibyte sequence

In [16]:
import csv
import json

def process_ingredient(ingredient_str):
    # JSON 형식의 배열로 변환
    ingredient_list = json.loads(ingredient_str.replace("'", '"'))
    return ingredient_list

def process_food_data(data_str):
    # JSON 형식의 배열로 변환
    data_list = json.loads(data_str.replace("'", '"'))
    return data_list

def csv_to_json(input_csv_path, output_json_path, encoding='utf-8'):
    # CSV 파일 열기
    with open(input_csv_path, 'r', encoding=encoding) as csv_file:
        # CSV 파일을 읽어오는 reader 생성
        csv_reader = csv.DictReader(csv_file)

        # JSON으로 변환할 데이터를 담을 리스트 생성
        data = []

        # CSV 데이터를 읽어서 리스트에 추가
        for row in csv_reader:
            # ingredient 필드를 JSON 형식의 배열로 변환
            row['ingredient'] = process_ingredient(row['ingredient'])

            # food_amount, food_unit, steps 컬럼을 JSON 형식의 배열로 변환
            row['food_amount'] = process_food_data(row['food_amount'])
            row['food_unit'] = process_food_data(row['food_unit'])
            row['steps'] = process_food_data(row['steps'])

            data.append(row)

    # JSON 파일로 쓰기
    with open(output_json_path, 'w', encoding='utf-8') as json_file:
        # JSON 파일에 데이터 쓰기
        json.dump(data, json_file, ensure_ascii=False, indent=2)

if __name__ == "__main__":
    # 변환할 CSV 파일 경로 및 JSON 파일 경로 설정
    input_csv_path = "/content/drive/MyDrive/wimf/LAST_MODI_food_dataframe11.csv"
    output_json_path = '/content/drive/MyDrive/wimf/LAST_MODI_food_dataframe11.json'

    # CSV를 JSON으로 변환 (만약 다른 인코딩이라면, encoding 매개변수를 해당 인코딩으로 변경)

    csv_to_json(input_csv_path, output_json_path, encoding='cp949')


JSONDecodeError: Expecting ',' delimiter: line 1 column 23 (char 22)

In [20]:
## 첫 성공 파일, 그러나 JSON 파일의 food_amount가 잘 나타나지 않는 경우가 발생하였음.

import csv
import json
import ast
import pandas as pd

def process_ingredient(ingredient_str):
    # 문자열에서 리스트로 변환
    return ast.literal_eval(ingredient_str) if pd.notna(ingredient_str) else []

def process_data(data_str):
    # 문자열에서 리스트로 변환
    try:
        return ast.literal_eval(data_str) if pd.notna(data_str) else []
    except (SyntaxError, ValueError):
        # SyntaxError 또는 ValueError가 발생할 경우 빈 리스트 반환
        return []

def csv_to_json(input_csv_path, output_json_path, encoding='utf-8'):
    # CSV 파일 열기
    with open(input_csv_path, 'r', encoding=encoding) as csv_file:
        # CSV 파일을 읽어오는 reader 생성
        csv_reader = csv.DictReader(csv_file)

        # JSON으로 변환할 데이터를 담을 리스트 생성
        data = []

        # CSV 데이터를 읽어서 리스트에 추가
        for row in csv_reader:
            # ingredient 필드를 문자열에서 리스트로 변환
            row['ingredient'] = process_ingredient(row['ingredient'])

            # food_amount, food_unit, steps 컬럼을 문자열에서 리스트로 변환
            row['food_amount'] = process_data(row['food_amount'])
            row['food_unit'] = process_data(row['food_unit'])
            row['steps'] = process_data(row['steps'])

            data.append(row)

    # JSON 파일로 쓰기
    with open(output_json_path, 'w', encoding='utf-8') as json_file:
        # JSON 파일에 데이터 쓰기
        json.dump(data, json_file, ensure_ascii=False, indent=2)

if __name__ == "__main__":
    # 변환할 CSV 파일 경로 및 JSON 파일 경로 설정
    input_csv_path = '/content/LAST_MODI_fdf.csv'
    output_json_path = '/content/LAST_MODI_fdf.json'

    # CSV를 JSON으로 변환 (만약 다른 인코딩이라면, encoding 매개변수를 해당 인코딩으로 변경)
    csv_to_json(input_csv_path, output_json_path, encoding='utf-8')


In [29]:
## 인코딩 오류르 해결하기 위한 코랩 내 변환

import pandas as pd

# CSV 파일 불러오기 (cp949로 인코딩된 파일 경로 지정)
input_csv_path = r'/content/LLASSTT_MODI_food_dataframe456456.csv'
df = pd.read_csv(input_csv_path, encoding='cp949')

# 새로운 CSV 파일로 저장 (utf-8로 인코딩)
output_csv_path = r'/content/LLASSTT_MODI_FDF.csv'
df.to_csv(output_csv_path, index=False, encoding='utf-8')

print(f"파일이 성공적으로 저장되었습니다: {output_csv_path}")


파일이 성공적으로 저장되었습니다: /content/LLASSTT_MODI_FDF.csv


In [30]:
##

import csv
import json
import ast
import pandas as pd

def process_ingredient(ingredient_str):
    # 문자열에서 리스트로 변환
    return ast.literal_eval(ingredient_str) if pd.notna(ingredient_str) else []

def process_data(data_str):
    # 문자열에서 리스트로 변환
    try:
        return ast.literal_eval(data_str) if pd.notna(data_str) else []
    except (SyntaxError, ValueError):
        # SyntaxError 또는 ValueError가 발생할 경우 빈 리스트 반환
        print(f"Error processing data string: {data_str}")
        return []

def csv_to_json(input_csv_path, output_json_path, encoding='utf-8', max_attempts=3):
    # CSV 파일 열기
    with open(input_csv_path, 'r', encoding=encoding) as csv_file:
        # CSV 파일을 읽어오는 reader 생성
        csv_reader = csv.DictReader(csv_file)

        # JSON으로 변환할 데이터를 담을 리스트 생성
        data = []

        # CSV 데이터를 읽어서 리스트에 추가
        for row in csv_reader:
            attempts = 0
            while attempts < max_attempts:
                try:
                    # ingredient 필드를 문자열에서 리스트로 변환
                    row['ingredient'] = process_ingredient(row['ingredient'])

                    # food_amount, food_unit, steps 컬럼을 문자열에서 리스트로 변환
                    row['food_amount'] = process_data(row['food_amount'])
                    row['food_unit'] = process_data(row['food_unit'])
                    row['steps'] = process_data(row['steps'])

                    data.append(row)
                    break
                except Exception as e:
                    print(f"Error processing row: {row}, Error: {e}")
                    attempts += 1

    # JSON 파일로 쓰기
    with open(output_json_path, 'w', encoding='utf-8') as json_file:
        # JSON 파일에 데이터 쓰기
        json.dump(data, json_file, ensure_ascii=False, indent=2)

if __name__ == "__main__":
    # 변환할 CSV 파일 경로 및 JSON 파일 경로 설정
    input_csv_path = '/content/LLASSTT_MODI_FDF.csv'
    output_json_path = '/content/LLASSTT_MODI_FDF.json'

    # CSV를 JSON으로 변환 (만약 다른 인코딩이라면, encoding 매개변수를 해당 인코딩으로 변경)
    csv_to_json(input_csv_path, output_json_path, encoding='utf-8')


In [25]:
import csv
import json
import ast
import pandas as pd
import re

def process_ingredient(ingredient_str):
    # 문자열에서 리스트로 변환
    return ast.literal_eval(ingredient_str) if pd.notna(ingredient_str) else []

def process_data(data_str):
    # 문자열에서 리스트로 변환
    try:
        # 유리수 형태인 경우 정규식을 활용하여 처리
        data = ast.literal_eval(data_str) if pd.notna(data_str) else []
        pattern = re.compile(r'(\d+)/(\d+)')
        return [Fraction(pattern.sub(r'\1/\2', str(item))).limit_denominator() if '/' in str(item) else item for item in data]
    except (SyntaxError, ValueError):
        # SyntaxError 또는 ValueError가 발생할 경우 빈 리스트 반환
        print(f"Error processing data string: {data_str}")
        return []

def csv_to_json(input_csv_path, output_json_path, encoding='utf-8', max_attempts=3):
    # CSV 파일 열기
    with open(input_csv_path, 'r', encoding=encoding) as csv_file:
        # CSV 파일을 읽어오는 reader 생성
        csv_reader = csv.DictReader(csv_file)

        # JSON으로 변환할 데이터를 담을 리스트 생성
        data = []

        # CSV 데이터를 읽어서 리스트에 추가
        for row in csv_reader:
            attempts = 0
            while attempts < max_attempts:
                try:
                    # ingredient 필드를 문자열에서 리스트로 변환
                    row['ingredient'] = process_ingredient(row['ingredient'])

                    # food_amount, food_unit, steps 컬럼을 문자열에서 리스트로 변환
                    row['food_amount'] = process_data(row['food_amount'])
                    row['food_unit'] = process_data(row['food_unit'])
                    row['steps'] = process_data(row['steps'])

                    data.append(row)
                    break
                except Exception as e:
                    print(f"Error processing row: {row}, Error: {e}")
                    attempts += 1

    # JSON 파일로 쓰기
    with open(output_json_path, 'w', encoding='utf-8') as json_file:
        # JSON 파일에 데이터 쓰기
        json.dump(data, json_file, ensure_ascii=False, indent=2)

if __name__ == "__main__":
    # 변환할 CSV 파일 경로 및 JSON 파일 경로 설정
    input_csv_path = '/content/LLASSTT_MODI_FDF.csv'
    output_json_path = '/content/LLASSTT_MODI_FDF.json'

    # CSV를 JSON으로 변환 (만약 다른 인코딩이라면, encoding 매개변수를 해당 인코딩으로 변경)
    csv_to_json(input_csv_path, output_json_path, encoding='utf-8')


Error processing data string: [1, 10, 2, 4, 1, 10, 1/2, 1/4, 1, None, 1]
Error processing data string: ['갈비는 물을 갈아가면서  2~3시간 정도 핏물을 빼주세요^~^', '그사이 양념을 만들어 주셔요~(T=밥수저 기준 입니다~!!) 간장10T,설탕2T,물엿4T,마늘 12쪽,대파10센티,양파1/2개,배1/4개,간파인애플1~2T,후추3번톡톡,참기름1T넣고 몽땅 갈아 주셔요~ (이때 맛을 보시고 개인취향에 맞겠금 단맛 짠맛을 가감해 주셔요^^)', '※파인애플을 넣어 주시면 연육작용 때문에 고기가 훨씬 부드러워 지지만 없으심 생략 하셔도 되셔요~ 절대 많이 넣으심 안되요~많이 넣으시면 고기가 키위 못지 않게 너덜너덜 해집니다^^; 배가 없으시면 사과를 대신 넣어 주시면 되구요~ 마늘이 넉넉히 들어가고 대파도 갈아 넣기 때문에 생강이랑 맛술 안넣으셔도 잡내가 하나도 안나요^~^ 핏물뺀 갈비는 물기를 제거해서~', '양념장을 골고루 뭍혀 냉장고에 2~3시간이상 재워 둡니다^~^', '달군 후라이팬에 양념이랑 같이 약불에서 구워 주시면 되셔요~ 저두 후라이팬에 구워 먹을라다가 오늘은 갈비는 숯불이라며 미니화로에 구웠어요^~^ 저처럼 숯불에 드시는거면 양념이 탈수 있기 때문에 양념을 조금 제거 하시고 구워 주셔요~^~^ (양념이 너무 맛나서 스테이크용 고기에도 급으로 양념발라 같이 구워 먹었네요ㅋㅋ)', '완성 사진은 굽자말자 먹느라 정신이 없어서;; 예전에 찍어뒀던 사진 올려요~ 한입먹는 순간 사진찍을 정신도 없게 만드는 맛이네요ㅎㅎ;;'       ]
Error processing data string: [2, 2, 1, 4, 3, 1/2, 1/2, 1/2, 2, None, None, None, None]
Error processing data string: ['가지를 손질해서 원하는 양 만큼 어슷 썰어줍니다. (어슷 썬 가지 2컵)', '팬을 달구기 전에 올리브유 4큰술과 다진파 1컵